# Datasets

Contains functions and classes related to datasets and dataloaders

In [ ]:
#| default_exp datasets

In [ ]:
#|export
from __future__ import annotations
import math,numpy as np,matplotlib.pyplot as plt
from operator import itemgetter
from itertools import zip_longest
import fastcore.all as fc

from torch.utils.data import default_collate

In [ ]:
import logging,pickle,gzip,os,time,shutil,torch,matplotlib as mpl
from pathlib import Path
import torch
from torch import tensor,nn,optim
from torch.utils.data import DataLoader
import torch.nn.functional as F
from datasets import load_dataset,load_dataset_builder

import torchvision.transforms.functional as TF
from fastcore.test import test_close, equals

In [ ]:
torch.set_printoptions(precision=2, linewidth=140, sci_mode=False)
torch.manual_seed(1)
mpl.rcParams['image.cmap'] = 'gray'

In [ ]:
logging.disable(logging.WARNING)

The dataset management will be developed and demonstrated using databases from Huggingface, but can be readily adapted to work with other datasets.  

## Hugging Face Datasets

Note that a list of the available datasets can be found on the Hugging Face hub: 

[Hugging Face Hub](https://huggingface.co/datasets)

Hugging Face datasets have a dictionary based structure


In [ ]:
name = "fashion_mnist"
ds_builder = load_dataset_builder(name)
print(ds_builder.info.description)

Fashion-MNIST is a dataset of Zalando's article images—consisting of a training set of
60,000 examples and a test set of 10,000 examples. Each example is a 28x28 grayscale image,
associated with a label from 10 classes. We intend Fashion-MNIST to serve as a direct drop-in
replacement for the original MNIST dataset for benchmarking machine learning algorithms.
It shares the same image size and structure of training and testing splits.



The dataset builder has a number of useful properties that can help better understand a dataset.  These are mostly sub properties the info attribute and include:
- features
- splits
- homepage

And lots of other useful things.

Dataset builder allows obtaining details about he dataset without actually downloading it.  Accessing the dataset then uses the load_dataet mehod

In [ ]:
ds_builder.info.features

{'image': Image(decode=True, id=None),
 'label': ClassLabel(names=['T - shirt / top', 'Trouser', 'Pullover', 'Dress', 'Coat', 'Sandal', 'Shirt', 'Sneaker', 'Bag', 'Ankle boot'], id=None)}

In [ ]:
ds_builder.info.splits

{'train': SplitInfo(name='train', num_bytes=31296655, num_examples=60000, shard_lengths=None, dataset_name=None),
 'test': SplitInfo(name='test', num_bytes=5233818, num_examples=10000, shard_lengths=None, dataset_name=None)}

In [ ]:
ds_builder.info.homepage

'https://github.com/zalandoresearch/fashion-mnist'

The load_dataset function can be used to load the whole dataset, which can then be sub divided if necessary

In [ ]:
ds_loader = load_dataset(name)

Extracting data files: 100%|████████████████████████████████████████████████████████████████| 4/4 [00:00<00:00, 380.82it/s]


Dataset fashion_mnist downloaded and prepared to /Users/johnrichmond/.cache/huggingface/datasets/fashion_mnist/fashion_mnist/1.0.0/0a671f063342996f19779d38c0ab4abef9c64f757b35af8134b331c294d7ba48. Subsequent calls will reuse this data.


100%|███████████████████████████████████████████████████████████████████████████████████████| 2/2 [00:00<00:00, 105.08it/s]


The different splits can then be assigned separately

In [ ]:
train_ds, test_ds = ds_loader['train'], ds_loader['test']

Retireving individual items can be done using the dataset features, in this case the image and label

In [ ]:
train_ds[0]

{'image': <PIL.PngImagePlugin.PngImageFile image mode=L size=28x28>,
 'label': 9}

In [ ]:
train_ds[0]['image']

This can be simplified by assigning the features (keys) to x and y

In [ ]:
x, y = ds_builder.info.features
x, y

('image', 'label')

In [ ]:
img = train_ds[0][x]
img

In [ ]:
#|export
def inplace(f):
    """ This function allows a function that does not return anything directly (ie one that modifies things
    without a return statenent) to then be used in an application that required a return.  So this function
    is a wrapper of another function that simply executes the function and then returns the modified input
    """
    def _f(b):
        f(b)
        return b
    return _f

Test that the inline function does as it should

In [ ]:
@inplace
def torch_cos(x):
    # Modify in place x to become cos(x)
    torch.cos_(x)

In [ ]:
angle = torch.tensor(torch.pi/8)
y = torch_cos(angle)
equals(y, torch.cos(torch.tensor(torch.pi/8)))

True

In [ ]:
#|export
def collate_dict(ds):
    """ when a dataset is defined by a dictionary this will identify the features and split into inputs and outputs
    as tensor arrays ready for input to a model
    """
    get = itemgetter(*ds.features)
    def _f(b):
        """ return a tuple containing the values associated with each of the keys returned by the itemgetter 
        given that default_collate return a dict with two keys and a stacked tensor for each """
        return get(default_collate(b))
    return _f

In [ ]:
#| hide
import nbdev; nbdev.nbdev_export()